In [ ]:
class Pokemon:
    def __init__(self, name=None, abilities=None, types=None, height=None, weight=None):
        self.name = name
        if abilities is None:
            self.abilities = []
        else:
            self.abilities = abilities
        if types is None:
            self.types = []
        else:
            self.types = types
        self.height = height
        self.weight = weight
        
    def from_dict(self, data):
        for field in ['name', 'abilities', 'types', 'height', 'weight']:
            if field in data:
                setattr(self, field, data[field])
                
    def __repr__(self):
        return f'<Pokemon: {self.name}>'
    
    def __str__(self):
        return f'{self.name.title()}'
        
class Pokedex:
    _list = []
    
    @classmethod
    def instructions(cls):
        print("""Type 'show' to view Pokedex.
Type 'quit' to exit the Pokedex.
Type 'sort' to view a categorized version of your Pokedex.""")
        
    @classmethod
    def show(cls):
        for idx, p in enumerate(cls._list):
            print(f'{idx+1}: {p}')
        
    @classmethod
    def populate(cls, pokemon_name):
        # check if the Pokemon we want to add has already been added to the Pokedex first
        
        # if we have any Pokemon in the Pokedex
        if cls._list:
            for p in cls._list:
                # if the Pokemon's name we passed into the function is equal to the name of a Pokemon
                # we've already created and stored in the Pokedex
                if pokemon_name.title() == p.name:
                    input("That pokemon already exists. Please try adding another... ")
        try:
            # connect to the API
            print('Please wait while we populate the Pokedex...')
            response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}').json()
            
            # create a new Pokemon
            p = Pokemon()
            data_dict = {
                'name': response['name'].title(),
                'abilities': [a['ability']['name'].title() for a in response['abilities']],
                'types': [t['type']['name'].title() for t in response['types']],
                'height': response['height'],
                'weight': response['weight']
            }
            p.from_dict(data_dict)
            
            # Add the pokemon to the Pokedex
            cls._list.append(p)
        except:
            input('There was an error populating your Pokedex. Try again. ')
            
    @classmethod
    def standardize(cls):
        # Build a Python dictionary that categorizes all Pokemon into any and ALL types they belong to
        standardized_dict = {}
        # loop into each player
        for p in cls._list:
            # while inside of the player, take a look at their tpyes list
            for t in p.types:
                # if the pokemon's type hasn't yet been found in the standardized dictionary
                if t not in standardized_dict:
                    # create a new key , with the key being the name of the type
                    # and set it as an empty dictionary
                    standardized_dict[t] = {}
        for p in cls._list:
            # if a Pokemon has multiples types or not
            for t in p.types:
                # Create a Pokemon dictionary that we will put into the Pokedex (cls._list)
                if p.name not in standardized_dict[t]:
                    poke_data = {
                        p.name: {
                            'abilities': p.abilities,
                            'height': p.height,
                            p.weight: 'weight'
                        }
                    }
                    standardized_dict[t].update(poke_data)
                else:
                    print('That Pokemon already exists')
        pp.pprint(standardized_dict)
    
    @classmethod
    def run(cls):
        done = False
        
        while not done:
            clear_output()
            
            cls.instructions()
            
            pokemon_name = input("Type in the name of a Pokemon to populate your Pokedex. Type 'quit' to quit program. ").lower()
            if pokemon_name == 'quit':
                done = True
            elif pokemon_name == 'show':
                cls.show()
                input('Press any key to continue. ')
            elif pokemon_name == 'sort':
                cls.standardize()
                input('Press any key to continue. ')
            else:
                Pokedex.populate(pokemon_name)
        
            
Pokedex.run()
